In [1]:
!pip install numpy scipy scikit-learn xgboost pandas torch 


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from pathlib import Path
import numpy as np
from scipy import stats
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from scipy.stats import skew
import xgboost as xgb
import pandas as pd
from scipy.stats import kurtosis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, f1_score
import xgboost as xgb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

#### Seeing how the dataset is structured

In [3]:
data_path = "data/"

In [4]:
df = pd.read_csv(data_path+"data_64Hz/S002_whole_df.csv")
df.head()

,TIMESTAMP,BVP,ACC_X,ACC_Y,ACC_Z,TEMP,EDA,HR,IBI,Sleep_Stage,Obstructive_Apnea,Central_Apnea,Hypopnea,Multiple_Events
0,0.000000,5.14,31.0,8.0,55.0,35.53,0.073005,49.0,NaN,P,NaN,NaN,NaN,NaN
1,0.015625,4.28,31.0,8.0,55.0,35.53,0.073005,49.0,NaN,P,NaN,NaN,NaN,NaN
2,0.031250,3.51,31.0,8.0,55.0,35.53,0.073005,49.0,NaN,P,NaN,NaN,NaN,NaN
3,0.046875,3.02,31.0,8.0,55.0,35.53,0.073005,49.0,NaN,P,NaN,NaN,NaN,NaN
4,0.062500,2.94,28.0,8.0,55.0,35.53,0.073005,49.0,NaN,P,NaN,NaN,NaN,NaN


### Checking missing values

In [5]:
df.isnull().sum()

TIMESTAMP                  0
BVP                        0
ACC_X                      0
ACC_Y                      0
ACC_Z                      0
TEMP                       0
EDA                        0
HR                         0
IBI                     1488
Sleep_Stage                0
Obstructive_Apnea    2004417
Central_Apnea        2013697
Hypopnea             1926605
Multiple_Events      2013697
dtype: int64

#### None of the columns that I am planning to use having missing values. Then, I will check the type of data of each column to check if further preprocessing is required for the columns.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2013697 entries, 0 to 2013696
Data columns (total 14 columns):
 #   Column             Dtype  
---  ------             -----  
 0   TIMESTAMP          float64
 1   BVP                float64
 2   ACC_X              float64
 3   ACC_Y              float64
 4   ACC_Z              float64
 5   TEMP               float64
 6   EDA                float64
 7   HR                 float64
 8   IBI                float64
 9   Sleep_Stage        object 
 10  Obstructive_Apnea  float64
 11  Central_Apnea      float64
 12  Hypopnea           float64
 13  Multiple_Events    float64
dtypes: float64(13), object(1)
memory usage: 215.1+ MB


#### After reading the notes for each column, every column is in the right data type. I will now check to see what values are in Sleep_stage since that is the target column. I also want to ensure strings like "missing", "no data", etc are not being recorded in the dataset. 

In [7]:
df["Sleep_Stage"].value_counts()

Sleep_Stage
N2    641280
P     587136
W     508801
R     153600
N1    122880
Name: count, dtype: int64

#### One of the files of the data is pretty reliable and are all in the correct format. I am going to assume that every other file is formatted the same way. However, I want to confirm if Sleep_stage and enough data exists for other patients' files. 

In [8]:
def visualize_all_participants(data_path):
    """
    Analyze and visualize all participant files without processing data
    
    Returns:
    participant_info: dict with analysis results for each participant
    """
    
    folder_path = os.path.join(data_path, "data_64Hz")
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    
    participant_info = {}
    
    # Initialize aggregation variables
    total_missing_data = {}
    total_sleep_stages = {}
    total_rows = 0
    total_sensor_data_points = 0
    total_missing_sensor_points = 0
    sensor_cols = ['BVP', 'ACC_X', 'ACC_Y', 'ACC_Z', 'TEMP', 'EDA', 'HR', 'IBI']
    
    print("="*60)
    print("PARTICIPANT DATA ANALYSIS")
    print("="*60)
    print(f"Processing {len(csv_files)} participant files...")
    
    for i, filename in enumerate(sorted(csv_files)):
        print(f"Processing {filename}... ({i+1}/{len(csv_files)})")
        
        # Read the file
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        
        # Aggregate basic info
        total_rows += len(df)
        
        # Aggregate missing values
        missing_data = df.isnull().sum()
        for col, missing_count in missing_data.items():
            if col not in total_missing_data:
                total_missing_data[col] = 0
            total_missing_data[col] += missing_count
        
        # Aggregate sleep stage counts
        sleep_stages = df["Sleep_Stage"].value_counts()
        for stage, count in sleep_stages.items():
            if stage not in total_sleep_stages:
                total_sleep_stages[stage] = 0
            total_sleep_stages[stage] += count
        
        # Aggregate sensor data points for quality calculation
        participant_sensor_points = len(df) * len(sensor_cols)
        participant_missing_sensor = missing_data[sensor_cols].sum()
        
        total_sensor_data_points += participant_sensor_points
        total_missing_sensor_points += participant_missing_sensor
        
        # Calculate individual data quality
        individual_quality = 1 - (participant_missing_sensor / participant_sensor_points)
        
        # Store participant info
        participant_info[filename] = {
            'shape': df.shape,
            'missing_data': missing_data.to_dict(),
            'sleep_stages': sleep_stages.to_dict(),
            'data_quality': individual_quality
        }
    
    # Calculate overall data quality
    overall_data_quality = 1 - (total_missing_sensor_points / total_sensor_data_points)
    
    # Display aggregated results
    print("\n" + "="*60)
    print("AGGREGATED RESULTS")
    print("="*60)
    
    print(f"Total participants analyzed: {len(csv_files)}")
    print(f"Total rows across all files: {total_rows:,}")
    
    print(f"\nTotal missing values (percentage of all data):")
    for col, missing_count in total_missing_data.items():
        total_col_data = total_rows  # Each column has total_rows data points
        missing_pct = (missing_count / total_col_data) * 100
        print(f"  {col}: {missing_count:,} ({missing_pct:.2f}%)")
    
    print(f"\nTotal sleep stage counts:")
    total_sleep_observations = sum(total_sleep_stages.values())
    for stage, count in sorted(total_sleep_stages.items()):
        percentage = (count / total_sleep_observations) * 100
        print(f"  {stage}: {count:,} ({percentage:.2f}%)")
    
    print(f"\nOverall data quality score: {overall_data_quality:.3f}")

In [9]:
visualize_all_participants(data_path)

PARTICIPANT DATA ANALYSIS
Processing 100 participant files...
Processing S002_whole_df.csv... (1/100)
Processing S003_whole_df.csv... (2/100)
Processing S004_whole_df.csv... (3/100)
Processing S005_whole_df.csv... (4/100)
Processing S006_whole_df.csv... (5/100)
Processing S007_whole_df.csv... (6/100)
Processing S008_whole_df.csv... (7/100)
Processing S009_whole_df.csv... (8/100)
Processing S010_whole_df.csv... (9/100)
Processing S011_whole_df.csv... (10/100)
Processing S012_whole_df.csv... (11/100)
Processing S013_whole_df.csv... (12/100)
Processing S014_whole_df.csv... (13/100)
Processing S015_whole_df.csv... (14/100)
Processing S016_whole_df.csv... (15/100)
Processing S017_whole_df.csv... (16/100)
Processing S018_whole_df.csv... (17/100)
Processing S019_whole_df.csv... (18/100)
Processing S020_whole_df.csv... (19/100)
Processing S021_whole_df.csv... (20/100)
Processing S022_whole_df.csv... (21/100)
Processing S023_whole_df.csv... (22/100)
Processing S024_whole_df.csv... (23/100)
Proc

#### There seems to be "Missing" string values in some patients' files in the Sleep_stage. Even though we have a a surplus of data windows, I still want to preserve data. My method of dealing with this is to fill out the missing windows with the sleep stage of the window before. I am assuming that the researchers who recorded the data did not record for this time frame because there were no changes to the patient and decided not to record. However, I did leave an alternative function to deal with this matter in case I wanted a different approach.

In [10]:
def deal_missing_windows(df):
    """
    Remove entire 30-second windows (1920 rows) that contain 'Missing' sleep stages
    """
    clean_windows = []
    
    for i in range(0, len(df), 1920):  # Every 1920 rows
        window = df.iloc[i:i+1920]
        
        # Check if this window has "Missing" sleep stage
        if 'Missing' not in window['Sleep_Stage'].values:
            clean_windows.append(window)
        else:
            print(f"Dropping window starting at row {i} due to 'Missing' sleep stage")
    
    if clean_windows:
        return pd.concat(clean_windows, ignore_index=True)
    else:
        return pd.DataFrame()  # Return empty DataFrame if no clean windows

In [11]:
def process_participants_to_arrays(data_path):
    """
    Process all participant files and return 3D arrays for training
    
    Returns:
    all_windowed_data: numpy array of shape (total_windows, 1920, 8) - all sensor data
    all_labels: numpy array of sleep stage labels for each window
    """
    
    folder_path = os.path.join(data_path, "data_64Hz")
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    
    all_windowed_data = []
    all_labels = []
    
    print("="*60)
    print("PROCESSING PARTICIPANTS TO 3D ARRAYS")
    print("="*60)
       
    for i, filename in enumerate(sorted(csv_files)):
        print(f"\n--- PROCESSING PARTICIPANT {i+1}: {filename} ---")
        
        # Read the file
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)

        # modifying the "P" stage as "W" since patient is technically awake
        df["Sleep_Stage"] = df["Sleep_Stage"].str.replace("P", "W")
        
        # Fill missing IBI values with 0 since most of the missing values are at the beginning - smooth out potential middle gaps with 
        # interpolation
        # the missing values at the beginning rows where interfering with the statistics that were needed for the basline model, XGBoost. 
        df['IBI'] = df['IBI'].fillna(0).interpolate(method='linear')

        
        # deal with windows with "Missing" sleep stages
        df_clean = deal_missing_windows(df)

        if len(df_clean) > 0:
            # Create windows
            n_complete_windows = len(df_clean) // 1920
            df_trimmed = df_clean.iloc[:n_complete_windows * 1920]
            
            # Extract sensor data
            sensor_cols = ['BVP', 'ACC_X', 'ACC_Y', 'ACC_Z', 'TEMP', 'EDA', 'HR', 'IBI']
            sensor_data = df_trimmed[sensor_cols].values
            windowed_data = sensor_data.reshape(n_complete_windows, 1920, len(sensor_cols))
            
            # Extract labels
            labels = df_trimmed['Sleep_Stage'].iloc[::1920].values
            
            # Add to collections
            all_windowed_data.append(windowed_data)
            all_labels.append(labels)
            
            print(f"  Windows created: {n_complete_windows}")
        else:
            print(f"  No clean windows available")

    # Combine all participants
    if all_windowed_data:
        all_windowed_data = np.concatenate(all_windowed_data, axis=0)
        all_labels = np.concatenate(all_labels, axis=0)
        
        print("\n" + "="*60)
        print("FINAL 3D DATASET")
        print("="*60)
        print(f"Total windows: {len(all_windowed_data)}")
        print(f"Data shape: {all_windowed_data.shape}")
        print(f"Labels shape: {all_labels.shape}")
        print(f"Unique sleep stages: {np.unique(all_labels)}")
        
        return all_windowed_data, all_labels
    else:
        print("No usable data found!")
        return None, None

### To create a flexible and effective dataset, I reformatted the panda dataframes into a 3D numpy array (# of total windows, 1920 - # of rows in a window, 8 - # of features). With this way of formatting, I can easily train my models and easily access the data as well. 

In [12]:
data, labels = process_participants_to_arrays(data_path)

PROCESSING PARTICIPANTS TO 3D ARRAYS

--- PROCESSING PARTICIPANT 1: S002_whole_df.csv ---
  Windows created: 1048

--- PROCESSING PARTICIPANT 2: S003_whole_df.csv ---
  Windows created: 1068

--- PROCESSING PARTICIPANT 3: S004_whole_df.csv ---
  Windows created: 1032

--- PROCESSING PARTICIPANT 4: S005_whole_df.csv ---
  Windows created: 1037

--- PROCESSING PARTICIPANT 5: S006_whole_df.csv ---
  Windows created: 1113

--- PROCESSING PARTICIPANT 6: S007_whole_df.csv ---
  Windows created: 1024

--- PROCESSING PARTICIPANT 7: S008_whole_df.csv ---
  Windows created: 1064

--- PROCESSING PARTICIPANT 8: S009_whole_df.csv ---
  Windows created: 1025

--- PROCESSING PARTICIPANT 9: S010_whole_df.csv ---
  Windows created: 1129

--- PROCESSING PARTICIPANT 10: S011_whole_df.csv ---
  Windows created: 1067

--- PROCESSING PARTICIPANT 11: S012_whole_df.csv ---
  Windows created: 1168

--- PROCESSING PARTICIPANT 12: S013_whole_df.csv ---
  Windows created: 1070

--- PROCESSING PARTICIPANT 13: S014

#### Building XGBBoost as my baseline model to compare my main model with. In this case, I chose the XGBoost model for several reasons. 
    1. I want to familiarize myself with more machine learning practices such as creating baseline model to compare my main model with. 
    2. I want to compare my CNN model with a model that is a suitable alternative to the dataset so I can evaulate the performance better. 
    3. XGBoost looks interesting and it is sort of an extension of decision trees - something that I am familiar with. However, I also wish to learn more machine learning models. It utilizes decisions trees but focuses on fixing the errors from previous trees in future trees. Works extremely well with tabular data and our dataset is perfect for it. 

In [13]:
def extract_features_from_window(window_data):
    """
    Extract statistics from each window frame and return it as an array of features. 
    
    window_data: shape(1920, 8) - one 30-second window
    returns: 1D array of statistical features from the window frame
    """
    features = []

    for sensor in range(window_data.shape[1]): 
        # Accessing each sensor and computing the statistics for each feature 
        data = window_data[:, sensor]

        # extend varies from append. extend adds to the list iteratively
        features.extend([
            np.mean(data),              # average 
            np.max(data),            # maximum
            np.min(data),           # minimum
            np.percentile(data, 25),    # 2nd quantile
            np.percentile(data, 75),   # 3rd quantile
            np.ptp(data),     # range
            np.var(data),     # variance
            np.std(data)      # standard deviation
        ])


        # there is potential to add more features here for feature specific statistics
        
    return np.array(features)

In [14]:
def create_feature_dataset(windowed_data):
    """
    Extracting statistics from 3D array to 2D array.  
    
    Args:
        windowed_data (array): 3D data of all windows with features 

    Returns:
        array: 2D array 
    """
    data = []
    for i, window in enumerate(windowed_data): 
        stats = extract_features_from_window(window)
        data.append(stats)
        if (i+1) % 1000 == 0: 
            print(f"Processed {i+1}/{len(windowed_data)} windows")

    return np.array(data)

In [ ]:
def train_xgboost_model(X_features, y): 
    """
    Training xgboost model using the features extracted from windows and labels as target.  

    Returns model evaluations from the XGBoost. Evaluation method includes accuracy and F1 scores. 
    """    
    # converting to labels 
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # splitting up data for training. ensuring stratification with sleep stages to keep class distribution across train and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X_features, y_encoded, test_size = 0.2, random_state = 42, stratify = y_encoded
    )  

    # scaling 
    scaler = StandardScaler() 
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # training parameters for XGBoost 
    model = xgb.XGBClassifier(
    objective= 'multi:softmax',   # multiple stages classification  
    eval_metric= 'mlogloss',           # picked mlogloss to showcase the model's 'confidence' of prediction for each window  
    n_estimators= 150,            # how many epochs of training
    learning_rate= 0.1,          # how much each tree contributes to final prediciton 
    max_depth= 7,               # how deep each tree can grow 
    subsample= 0.8,            # % of data used in training
    colsample_bytree= 0.8     # % of features used in training 
    )
    # training the model 
    model.fit(X_train_scaled, y_train)

    # making predictions 
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)

    # evaluation 
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    f1_macro = f1_score(y_test, y_test_pred, average = "macro")
    f1_weighted = f1_score(y_test, y_test_pred, average = "weighted")

    print("MODEL EVALUATION")
    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Macro F1-Score: {f1_macro:.4f}")
    print(f"Weighted F1-Score: {f1_weighted:.4f}")
    print("-" * 50)
    print("Classification Report")
    print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_))

In [16]:
X_features = create_feature_dataset(data)

Processed 1000/105891 windows
Processed 2000/105891 windows
Processed 3000/105891 windows
Processed 4000/105891 windows
Processed 5000/105891 windows
Processed 6000/105891 windows
Processed 7000/105891 windows
Processed 8000/105891 windows
Processed 9000/105891 windows
Processed 10000/105891 windows
Processed 11000/105891 windows
Processed 12000/105891 windows
Processed 13000/105891 windows
Processed 14000/105891 windows
Processed 15000/105891 windows
Processed 16000/105891 windows
Processed 17000/105891 windows
Processed 18000/105891 windows
Processed 19000/105891 windows
Processed 20000/105891 windows
Processed 21000/105891 windows
Processed 22000/105891 windows
Processed 23000/105891 windows
Processed 24000/105891 windows
Processed 25000/105891 windows
Processed 26000/105891 windows
Processed 27000/105891 windows
Processed 28000/105891 windows
Processed 29000/105891 windows
Processed 30000/105891 windows
Processed 31000/105891 windows
Processed 32000/105891 windows
Processed 33000/1

In [17]:
train_xgboost_model(X_features, labels)

MODEL EVALUATION
Training Accuracy: 0.8590
Test Accuracy: 0.8016
Macro F1-Score: 0.6793
Weighted F1-Score: 0.7817
--------------------------------------------------
Classification Report
              precision    recall  f1-score   support

          N1       0.57      0.14      0.23      1765
          N2       0.74      0.90      0.81      8011
          N3       0.86      0.68      0.76       541
           R       0.87      0.62      0.72      1680
           W       0.86      0.89      0.87      9182

    accuracy                           0.80     21179
   macro avg       0.78      0.64      0.68     21179
weighted avg       0.79      0.80      0.78     21179



#### Several key insights can be drawn from the classification report of the XGBoost model. For instance, I was able to see the class imbalance between the sleep stages with the N3 (deep sleep) having the least amount of data. With more research, I learned that N3 is also the shortest period of sleep stage during the night for an average person. Despite having fewer samples than N1, N3 achieved better results - this could be explained by deep sleep having more distinctive physiological patterns that are easier for the model to recognize. In contrast, N1 performed poorly with only 8% recall, likely because it's a transitional stage between wake and sleep, making the patterns less clear. From this baseline model, I will know how to approach the dataset for my main model.  

#### My average accuracy score for XGBoost is somewhere around 75% to 81% depending on the random state. I will use this as a benchmark to see if my CNN model can beat it. However, this accuracy score as a baseline can be quite difficult to beat since XGBoost is a very powerful model.

## I will now begin building my 1D CNN model. 

### Data Preparation

In [45]:
# Fix the data preparation section
label_encoder = LabelEncoder() 
y_encoded = label_encoder.fit_transform(labels)

# Reformatting numpy arrays to fit PyTorch's preferred format 
X_pytorch = np.transpose(data, (0, 2, 1))

# First split: train vs test (60% train, 40% test)
X_train, X_test, y_train, y_test = train_test_split(
    X_pytorch, y_encoded, 
    test_size=0.4, 
    random_state=42, 
    stratify=y_encoded
)

# Second split: split train into train + validation (48% train, 12% val, 40% test)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, 
    test_size=0.2,  
    random_state=42, 
    stratify=y_train
)

# Reshape for StandardScaler
n_train, channels, length = X_train.shape
X_train_reshaped = X_train.reshape(n_train, -1)
X_val_reshaped = X_val.reshape(X_val.shape[0], -1)
X_test_reshaped = X_test.reshape(X_test.shape[0], -1)

# Scaling 
scaler = StandardScaler() 
X_train_scaled = scaler.fit_transform(X_train_reshaped)
X_val_scaled = scaler.transform(X_val_reshaped)
X_test_scaled = scaler.transform(X_test_reshaped)

# Reshape back to 3D
X_train_scaled = X_train_scaled.reshape(n_train, channels, length)
X_val_scaled = X_val_scaled.reshape(X_val.shape[0], channels, length)
X_test_scaled = X_test_scaled.reshape(X_test.shape[0], channels, length)

# Convert to tensors
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_train_tensor = torch.from_numpy(X_train_scaled).float()
X_val_tensor = torch.from_numpy(X_val_scaled).float()
y_train_tensor = torch.from_numpy(y_train).long()
y_val_tensor = torch.from_numpy(y_val).long()

# Create datasets and dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print(f"Train set: {X_train_tensor.shape}")
print(f"Validation set: {X_val_tensor.shape}")
print("DataLoaders created successfully!")

Train set: torch.Size([50827, 8, 1920])
Validation set: torch.Size([12707, 8, 1920])
DataLoaders created successfully!


In [46]:
class SleepCNN(nn.Module): # drawing traits from nn.Module
    def __init__(self):
        super(SleepCNN, self).__init__() # setting up Pytorch infrastructure 
        
        self.conv1 = nn.Conv1d(8, 16, kernel_size=11, padding=5)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(4)
        
        self.conv2 = nn.Conv1d(16, 32, kernel_size=7, padding=3)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(4)
        
        # After pooling: 1920 -> 480 -> 120
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(32 * 120, 64)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(64, 5)
    
    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.flatten(x)
        x = self.dropout(x)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

### Intializing the environment and model and testing if everything works properly

In [47]:
model = SleepCNN()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Test with dummy data
test_input = torch.randn(4, 8, 1920).to(device)
output = model(test_input)
print(f"Input shape: {test_input.shape}")
print(f"Output shape: {output.shape}")
print(f"Model parameters: {sum(p.numel() for p in model.parameters())}")

Input shape: torch.Size([4, 8, 1920])
Output shape: torch.Size([4, 5])
Model parameters: 251189


## Setting up the loss function and optimizer

In [48]:
# Set up training components
criterion = nn.CrossEntropyLoss()  # Multi-class classification loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)  # Adam optimizer with learning rate 0.001

# Move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(f"Training on: {device}")

Training on: cpu


### Training

In [49]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20):
    train_losses = []
    val_losses = []
    val_accuracies = []
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()  # Enable dropout and batch norm training mode
        running_loss = 0.0
        
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            optimizer.zero_grad()        # Reset gradients
            outputs = model(batch_X)     # Forward pass
            loss = criterion(outputs, batch_y)  # Calculate loss
            loss.backward()              # Backward pass
            optimizer.step()             # Update weights
            
            running_loss += loss.item()
        
        # Validation phase
        model.eval()  # Disable dropout, enable evaluation mode
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():  # Disable gradient computation for efficiency
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                val_loss += loss.item()
                
                _, predicted = torch.max(outputs, 1)  # Get predicted classes
                total += batch_y.size(0)
                correct += (predicted == batch_y).sum().item()
        
        # Calculate averages
        avg_train_loss = running_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * correct / total
        
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        val_accuracies.append(val_accuracy)
        
        print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')
        print('-' * 60)
    
    return train_losses, val_losses, val_accuracies

### Running Training

In [50]:
# Train your model
print("Starting CNN training...")
train_losses, val_losses, val_accuracies = train_model(
    model, train_loader, val_loader, criterion, optimizer, num_epochs=20
)

# Check final validation accuracy
final_val_acc = val_accuracies[-1]
print(f"\nFinal Validation Accuracy: {final_val_acc:.2f}%")
print(f"XGBoost Baseline: 80.00%")
if final_val_acc > 80:
    print("CNN beats XGBoost baseline!")
else:
    print("CNN needs improvement to beat baseline.")

Starting CNN training...
Epoch [1/20]
Train Loss: 1.0818, Val Loss: 1.0300, Val Acc: 60.35%
------------------------------------------------------------
Epoch [2/20]
Train Loss: 1.0102, Val Loss: 0.9923, Val Acc: 62.61%
------------------------------------------------------------
Epoch [3/20]
Train Loss: 0.9692, Val Loss: 0.9598, Val Acc: 63.26%
------------------------------------------------------------
Epoch [4/20]
Train Loss: 0.9420, Val Loss: 0.9497, Val Acc: 64.03%
------------------------------------------------------------
Epoch [5/20]
Train Loss: 0.9179, Val Loss: 0.9183, Val Acc: 64.65%
------------------------------------------------------------
Epoch [6/20]
Train Loss: 0.8966, Val Loss: 0.9220, Val Acc: 65.42%
------------------------------------------------------------
Epoch [7/20]
Train Loss: 0.8715, Val Loss: 0.9169, Val Acc: 64.82%
------------------------------------------------------------
Epoch [8/20]
Train Loss: 0.8547, Val Loss: 0.9016, Val Acc: 65.45%
------------